# Overview: Generating Model Grids

This notebook provides an example of how to generate a custom grid of models within `brutus`, for us as part of the `fitting` module. More detailed information can be found in the documentation and in other notebooks. Files needed to run various parts of the code can also be found online.

In [1]:
import brutus
from brutus import seds

New grids can be generated using the `SEDmaker` class, which requires an input **neural network (NN) file** to generate photometry and **equivalent evolutionary point (EEP) track file** to generate the stellar models. These can also take in particular filters from the `filters` module. If no filters are passed, photometry is by default generated over all available bands.

In [2]:
# initialize the SED maker for the MIST v1.2 models
mist = seds.SEDmaker(nnfile='../data/DATAFILES/nn_c3k.h5',
                     mistfile='../data/DATAFILES/MIST_1.2_EEPtrk.h5')

Filters: ['Gaia_G_MAW', 'Gaia_BP_MAWf', 'Gaia_RP_MAW', 'SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z', 'PS_g', 'PS_r', 'PS_i', 'PS_z', 'PS_y', 'PS_w', 'PS_open', 'DECam_u', 'DECam_g', 'DECam_r', 'DECam_i', 'DECam_z', 'DECam_Y', 'Bessell_U', 'Bessell_B', 'Bessell_V', 'Bessell_R', 'Bessell_I', '2MASS_J', '2MASS_H', '2MASS_Ks', 'VISTA_Z', 'VISTA_Y', 'VISTA_J', 'VISTA_H', 'VISTA_Ks', 'UKIDSS_Z', 'UKIDSS_Y', 'UKIDSS_J', 'UKIDSS_H', 'UKIDSS_K', 'WISE_W1', 'WISE_W2', 'WISE_W3', 'WISE_W4', 'Tycho_B', 'Tycho_V', 'Hipparcos_Hp', 'Kepler_D51', 'Kepler_Kp', 'TESS']
Constructing MIST library...done!
Computing age weights for track (mini, feh, afe) = (300.0, 0.5, 0.0)            
Initializing FastNN predictor...done!


Generating the grid is as simple as running the `make_grid` function. This can generate a grid over initial mass (`mini`), initial metallicity (`feh`), EEP (`eep`), alpha-element abundance variation (`afe`), and secondary mass fraction (`smf`). Note that while defaults are specified, the function is designed with the expectation that users will pass at least some additional arguments. For additional information on the available options, please see the release paper and the documentation.

In [3]:
# build the SED grid
mist.make_grid(smf_grid=np.array([0.]),  # no binaries
               afe_grid=np.array([0.]))  # no afe

Constructing grid 100.000% (818620/818620) [mini= 2.000, eep=808.000, feh= 0.500, afe= 0.000, smf= 0.000] (t/obj: 18.459 ms, est. remaining:      0.000 s)          


The output grid is saved internally, with the grid labels (`grid_label`), output stellar parameters (`grid_param`), and spectral energy distributions (SEDs) (`grid_sed`) saved. Since not all models on the grid are well-defined (i.e. there are no models for evolved low-mass stars since they are distinctly unphysical), there is also a selection array (`grid_sel`) created.

For compatibility with the `load_models` function in the `utils` module, the models have to saved to disk in hdf5 format with the internal structure shown below.

In [4]:
# dump results to disk
import h5py

grid_vers = 'v9'  # version number of MIST grid

with h5py.File("../data/grid_mist_{}.h5".format(grid_vers), "w") as out:
    
    # selection array
    sel = mist.grid_sel
    
    # labels used to generate the grid
    labels = out.create_dataset("labels", data=mist.grid_label[sel])
    
    # parameters generated interpolating over the MIST isochrones
    pars = out.create_dataset("parameters", data=mist.grid_param[sel])
    
    # SEDS generated using the NN from the stellar parameters
    seds = out.create_dataset("mag_coeffs", data=mist.grid_sed[sel])

And we're done! We can now use this stellar model grid in other parts of the code.